# Introduction
This notebook shows how to develop a Discord bot that will publish signals generated by a strategy to a Discord channel. This notebook assumes you already have created a token that allows the bot to publish messages to the channel.

The library used to integrate with Discord is `JDA`. It is a feature rich library and this notebook only shows the basics of what is possible. For more info visit https://github.com/DV8FromTheWorld/JDA.

WARNING: a bot can quickly generate many messages and flood a Discord channel. So be very careful when experimenting with this notebook using existing message channels. 

In [2]:
// Load the JDA library and import the required classes
@file:DependsOn("net.dv8tion:JDA:5.5.1")
import net.dv8tion.jda.api.*
import net.dv8tion.jda.api.entities.MessageEmbed
import net.dv8tion.jda.api.entities.channel.middleman.MessageChannel
import net.dv8tion.jda.api.utils.messages.MessageCreateData


In [3]:
%use roboquant(version=3.0.0-SNAPSHOT, modules=alpaca)

import org.roboquant.alpaca.*
Welcome()

roboquant 3.0.0-SNAPSHOT 
 build: 2025-05-06T06:39:03Z 
 home: /Users/peter/.roboquant 
 os: Mac OS X 15.4.1 
 jvm: OpenJDK 64-Bit Server VM 17.0.15 
 memory: 3256 MB 
 cpu cores: 8

## DiscordBot
This is the implementation of the bot itself that publishes a signal to the Discord channel. 

When creating an instance of `DiscordBot`, it will authenticate itself using an API token. Please note although it is possible to include the token in the code, it is better practice to define the token outside the code so it doesn't leak to the public by mistake. See also the roboquant documentation how to define these properties.

After being authenticated, it will get the channel based on the provided channel name. If there are more channels wit the same name, it will used the first one.

In [4]:
class DiscordBot(val channelName:String = "general")  {

    private val jda: JDA
    private val channel: MessageChannel

    init {
        // val token = "my_secret_discord_token"
        val token = Config.getProperty("discord.token")!!
        jda = JDABuilder
            .createDefault(token)
            .build()
            .awaitReady()

        channel = jda.getTextChannelsByName(channelName, true).first()
    }

 
    fun publish(signal: Signal, time: Instant) {

        val fields = listOf(
            MessageEmbed.Field("asset", signal.asset.symbol, true),
            MessageEmbed.Field("rating", signal.rating.toString(), true),
            MessageEmbed.Field("time", time.toString(), true)
        )

        val embed = EmbedBuilder()
            .setTitle("Signal Detector")
            .setAuthor("roboquant", "http://roboquant.org", "http://roboquant.org/img/avatar.png")
            .setFooter("generated by roboquant","http://roboquant.org/img/avatar.png")

        embed.fields.addAll(fields)
        val msg = MessageCreateData.fromEmbeds(embed.build())
        channel.sendMessage(msg).queue()
    }

}

## DiscordTrader
Normally a trader in roboquant receives signals generated by a strategy and converts them to orders that are then send to a broker.

But in this scenario, we just publish the received signals to a Discord channel and don't generate any orders.      

In [5]:
class DiscordTrader(private val bot: DiscordBot) : org.roboquant.traders.Trader {
    
    override fun createOrders(signals: List<Signal>, account: Account, event: Event): List<Order> {
        for (signal in signals) bot.publish(signal, event.time)
        return emptyList()
    }

}

## Data feed

For this notebook we use live market data from Alpaca. We subscribe to ETH/BUSD and BTC/BUSD candlesticks

In [7]:
val feed = AlpacaLiveFeed()
feed.subscribeCrypto("ETHBUSD", "BTCBUSD")

couldn't establish net.jacobpeterson.alpaca.websocket.marketdata.streams.stock.StockMarketDataWebsocket@2426eb22 
 org.roboquant.common.ConfigurationException: couldn't establish net.jacobpeterson.alpaca.websocket.marketdata.streams.stock.StockMarketDataWebsocket@2426eb22
	at org.roboquant.alpaca.AlpacaLiveFeed.connectMarket(AlpacaLiveFeed.kt:107)
	at org.roboquant.alpaca.AlpacaLiveFeed.connect(AlpacaLiveFeed.kt:93)
	at org.roboquant.alpaca.AlpacaLiveFeed.<init>(AlpacaLiveFeed.kt:84)
	at org.roboquant.alpaca.AlpacaLiveFeed.<init>(AlpacaLiveFeed.kt:73)
	at Line_17_jupyter.<init>(Line_17.jupyter.kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:122)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:48)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnostics$1.invokeSuspend(InternalEvaluatorImpl.kt:137)
	at kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationImpl.kt:33)
	at kotlinx.coroutines.DispatchedTask.run(DispatchedTask.kt:104)
	at kotlinx.coroutines.EventLoopImplBase.processNextEvent(EventLoop.common.kt:277)
	at kotlinx.coroutines.BlockingCoroutine.joinBlocking(Builders.kt:95)
	at kotlinx.coroutines.BuildersKt__BuildersKt.runBlocking(Builders.kt:69)
	at kotlinx.coroutines.BuildersKt.runBlocking(Unknown Source)
	at kotlinx.coroutines.BuildersKt__BuildersKt.runBlocking$default(Builders.kt:48)
	at kotlinx.coroutines.BuildersKt.runBlocking$default(Unknown Source)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:137)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:80)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:78)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.withHost(ReplForJupyterImpl.kt:791)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute-L4Nmkdk(CellExecutorImpl.kt:78)
	at org.jetbrains.kotlinx.jupyter.repl.execution.CellExecutor$DefaultImpls.execute-L4Nmkdk$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.evaluateUserCode-wNURfNM(ReplForJupyterImpl.kt:613)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.evalExImpl(ReplForJupyterImpl.kt:471)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.access$evalExImpl(ReplForJupyterImpl.kt:143)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl$evalEx$1.invoke(ReplForJupyterImpl.kt:464)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl$evalEx$1.invoke(ReplForJupyterImpl.kt:463)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.withEvalContext(ReplForJupyterImpl.kt:444)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.evalEx(ReplForJupyterImpl.kt:463)
	at org.jetbrains.kotlinx.jupyter.messaging.IdeCompatibleMessageRequestProcessor$processExecuteRequest$1$response$1$1.invoke(IdeCompatibleMessageRequestProcessor.kt:159)
	at org.jetbrains.kotlinx.jupyter.messaging.IdeCompatibleMessageRequestProcessor$processExecuteRequest$1$response$1$1.invoke(IdeCompatibleMessageRequestProcessor.kt:158)
	at org.jetbra

## Run the bot
Now we create an instance of Roboquant with the Strategy we wan't to use to generate the signals and we run it for a period of time. 

So every time the strategy has generated a signal during this time, a new message will be publised to the Discord channel. 

In [9]:
val bot = DiscordBot("bot-insights")
val trader = DiscordTrader(bot)
val strategy = EMACrossover()

run(feed, strategy, trader = trader, timeframe = Timeframe.next(4.hours))

List is empty. 
 java.util.NoSuchElementException: List is empty.
	at kotlin.collections.CollectionsKt___CollectionsKt.first(_Collections.kt:214)
	at Line_14_jupyter$DiscordBot.<init>(Line_14.jupyter.kts:14)
	at Line_19_jupyter.<init>(Line_19.jupyter.kts:1)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:122)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:48)
	at kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
	at kotlin.script.experimental.jvm.BasicJvmReplEvaluator.eval(BasicJvmReplEvaluator.kt:49)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl$eval$resultWithDiagnostics$1.invokeSuspend(InternalEvaluatorImpl.kt:137)
	at kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationImpl.kt:33)
	at kotlinx.coroutines.DispatchedTask.run(DispatchedTask.kt:104)
	at kotlinx.coroutines.EventLoopImplBase.processNextEvent(EventLoop.common.kt:277)
	at kotlinx.coroutines.BlockingCoroutine.joinBlocking(Builders.kt:95)
	at kotlinx.coroutines.BuildersKt__BuildersKt.runBlocking(Builders.kt:69)
	at kotlinx.coroutines.BuildersKt.runBlocking(Unknown Source)
	at kotlinx.coroutines.BuildersKt__BuildersKt.runBlocking$default(Builders.kt:48)
	at kotlinx.coroutines.BuildersKt.runBlocking$default(Unknown Source)
	at org.jetbrains.kotlinx.jupyter.repl.impl.InternalEvaluatorImpl.eval(InternalEvaluatorImpl.kt:137)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:80)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl$execute$1$result$1.invoke(CellExecutorImpl.kt:78)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.withHost(ReplForJupyterImpl.kt:791)
	at org.jetbrains.kotlinx.jupyter.repl.impl.CellExecutorImpl.execute-L4Nmkdk(CellExecutorImpl.kt:78)
	at org.jetbrains.kotlinx.jupyter.repl.execution.CellExecutor$DefaultImpls.execute-L4Nmkdk$default(CellExecutor.kt:13)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.evaluateUserCode-wNURfNM(ReplForJupyterImpl.kt:613)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.evalExImpl(ReplForJupyterImpl.kt:471)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.access$evalExImpl(ReplForJupyterImpl.kt:143)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl$evalEx$1.invoke(ReplForJupyterImpl.kt:464)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl$evalEx$1.invoke(ReplForJupyterImpl.kt:463)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.withEvalContext(ReplForJupyterImpl.kt:444)
	at org.jetbrains.kotlinx.jupyter.repl.impl.ReplForJupyterImpl.evalEx(ReplForJupyterImpl.kt:463)
	at org.jetbrains.kotlinx.jupyter.messaging.IdeCompatibleMessageRequestProcessor$processExecuteRequest$1$response$1$1.invoke(IdeCompatibleMessageRequestProcessor.kt:159)
	at org.jetbrains.kotlinx.jupyter.messaging.IdeCompatibleMessageRequestProcessor$processExecuteRequest$1$response$1$1.invoke(IdeCompatibleMessageRequestProcessor.kt:158)
	at org.jetbrains.kotlinx.jupyter.streams.BlockingSubstitutionEngine.withDataSubstitution(SubstitutionEngine.kt:70)
	at org.jetbrains.kotlinx.jupyter.streams.StreamSubstitutionManager.withSubstitutedStreams(StreamSubstitutionManager.kt:118)
	at org.jetbrains.kotlinx.jupyter.messaging.IdeCompatibleMessageRequestProcessor.withForkedIn(IdeCompatibleMessageRequestPr